In [1]:
import os, cv2
import numpy as np

def read_anno_file(anno_file):
    assert os.path.exists(anno_file), "Annotation file does not exist!" + anno_file
    result = []
    with open(anno_file, 'r') as f:
        for line in f.readlines():
            items = {}
            items['vid'] = line.strip().split(',[')[0]
            labels = line.strip().split(',[')[1].split('],')[0]
            items['label'] = [int(val) for val in labels.split(',')]
            others = line.strip().split(',[')[1].split('],')[1].split(',')
            items['startframe'], items['vid_ytb'], items['lighting'], items['weather'], items['ego_involve'] = others
            result.append(items)
    f.close()
    return result


def get_video_frames(video_file, topN=50):
    # get the video data
    cap = cv2.VideoCapture(video_file)
    ret, frame = cap.read()
    video_data = []
    while (ret):
        video_data.append(frame)
        ret, frame = cap.read()
    print("original # frames: %d"%(len(video_data)))
    assert len(video_data) >= topN
    video_data = video_data[:topN]
    return video_data



In [298]:

""""if __name__ == "__main__":
    anno_file = "../videos/Crash-1500.txt"
    anno_data = read_anno_file(anno_file)

    video_path = "../videos/Crash-1500"
    for anno in anno_data:
        video_file = os.path.join(video_path, anno['vid'] + ".mp4")
        assert os.path.exists(video_file), "video file does not exist!" + video_file
        # read frames
        frames = get_video_frames(video_file, topN=50)
        labels = anno['label']
        print("file: %s, # frames: %d, # labels: %d."%(video_file, len(frames), len(labels)))
        print(len(labels))
        for idx, im in enumerate(frames):
           if labels[idx] == 1:
               cv2.putText(im, 'Accident', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
           else:
               cv2.putText(im, 'Normal', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 2, cv2.LINE_AA)
           cv2.imshow('frame', im)
           cv2.waitKey(100)"""""

'"if __name__ == "__main__":\n    anno_file = "../videos/Crash-1500.txt"\n    anno_data = read_anno_file(anno_file)\n\n    video_path = "../videos/Crash-1500"\n    for anno in anno_data:\n        video_file = os.path.join(video_path, anno[\'vid\'] + ".mp4")\n        assert os.path.exists(video_file), "video file does not exist!" + video_file\n        # read frames\n        frames = get_video_frames(video_file, topN=50)\n        labels = anno[\'label\']\n        print("file: %s, # frames: %d, # labels: %d."%(video_file, len(frames), len(labels)))\n        print(len(labels))\n        for idx, im in enumerate(frames):\n           if labels[idx] == 1:\n               cv2.putText(im, \'Accident\', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)\n           else:\n               cv2.putText(im, \'Normal\', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 2, cv2.LINE_AA)\n           cv2.imshow(\'frame\', im)\n           cv2.wa

In [2]:
annotation = read_anno_file("Dataset/CarCrash/videos/Crash-1500.txt")

In [3]:
import pandas as pd
annotation_df=pd.DataFrame(annotation)

la colonne label contiens 50 label qui sont des booléens. 1 donne la frame ou se situe l'accident.

In [4]:
annotation_df.head()

,vid,label,startframe,vid_ytb,lighting,weather,ego_involve
0,000001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000285,0000,Day,Normal,Yes
1,000002,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000646,0000,Day,Normal,Yes
2,000003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000972,0000,Day,Normal,Yes
3,000004,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",001257,0000,Day,Normal,Yes
4,000005,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000462,0010,Day,Snowy,Yes


In [5]:
def get_accident_frame(row):
    return row["label"].index(1)

annotation_df["accident_frame"]=annotation_df.apply(get_accident_frame,axis=1)


In [6]:
annotation_df["accident_frame"].describe()

count    1500.000000
mean       37.189333
std         5.787557
min        30.000000
25%        32.000000
50%        36.000000
75%        42.000000
max        49.000000
Name: accident_frame, dtype: float64

In [7]:
annotation_df.head()

,vid,label,startframe,vid_ytb,lighting,weather,ego_involve,accident_frame
0,000001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000285,0000,Day,Normal,Yes,32
1,000002,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000646,0000,Day,Normal,Yes,30
2,000003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000972,0000,Day,Normal,Yes,31
3,000004,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",001257,0000,Day,Normal,Yes,41
4,000005,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",000462,0010,Day,Snowy,Yes,30


In [8]:
annotation_df.ego_involve.value_counts()

ego_involve
Yes    801
No     699
Name: count, dtype: int64

On fait le choix de ne garder que les voitures qui sont elles même dans un accident.

In [9]:
annotation_df_reduced = annotation_df[annotation_df.ego_involve=="Yes"]

annotation_df_reduced.shape

(801, 8)

In [10]:
annotation_df_reduced=annotation_df[["vid","label","accident_frame","ego_involve"]]
annotation_df_reduced["accident"]=1

/var/folders/88/07qx0wt566bgy_rrcgckgl0r0000gn/T/ipykernel_96710/2177464081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_df_reduced["accident"]=1


In [11]:
annotation_df_reduced.head()

,vid,label,accident_frame,ego_involve,accident
0,000001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",32,Yes,1
1,000002,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",30,Yes,1
2,000003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",31,Yes,1
3,000004,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",41,Yes,1
4,000005,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",30,Yes,1


In [12]:
def read_features_file(file):
    #Code to open and read feature in npz files
    features = np.load(file)    
    return features
    

- data: Extracted 4096-dim features with shape (50, 20, 4096). It contains frame-level feature with shape (50, 1, 4096) and 19 box-level features with shape (50, 19, 4096).
- det: Detected bounding boxes with shape (50, 19, 6), where the last dim denotes (x1, y1, x2, y2, prob, cls).
- labels: One-hot video labels to indicate whether the video contains an accident, i.e., [0, 1] denotes positive (accident) and [1, 0] denotes negative (normal).
- ID: The video name for current feature file.


In [13]:
np_file=read_features_file("Dataset/CarCrash/vgg16_features/positive/000004.npz")

In [14]:
np_file.files

['data', 'det', 'labels', 'ID']

In [15]:
np_file["labels"].shape

(2,)

In [16]:
np_file["labels"]

array([0, 1])

In [17]:
df_test=pd.read_csv("Dataset/CarCrash/vgg16_features/test.txt", sep=" ", header=None)
df_test.columns=["npz_file","label"]
df_test.head()

,npz_file,label
0,positive/000353.npz,1
1,positive/000849.npz,1
2,negative/001320.npz,0
3,negative/000210.npz,0
4,negative/002373.npz,0


In [18]:
df_train=pd.read_csv("Dataset/CarCrash/vgg16_features/train.txt", sep=" ", header=None)
df_train.columns=["npz_file","label"]
df_train.head()


,npz_file,label
0,negative/001355.npz,0
1,negative/002192.npz,0
2,negative/002082.npz,0
3,negative/000754.npz,0
4,positive/000713.npz,1


In [19]:
display(df_train.shape)
df_test.shape

(3600, 2)

(900, 2)

In [20]:
df=pd.concat([df_train,df_test])

In [21]:
df["video_file"]=df["npz_file"].apply(lambda x: x.split("/")[-1].split(".")[0])

df.head()


,npz_file,label,video_file
0,negative/001355.npz,0,001355
1,negative/002192.npz,0,002192
2,negative/002082.npz,0,002082
3,negative/000754.npz,0,000754
4,positive/000713.npz,1,000713


In [22]:
df["video_path"] = df.apply(lambda x: "Dataset/CarCrash/videos/Crash-1500/" + x["video_file"] + ".mp4" if x["label"] == 1 else "Dataset/CarCrash/videos/Normal/" + x["video_file"] + ".mp4", axis=1)

df.head()   

,npz_file,label,video_file,video_path
0,negative/001355.npz,0,001355,Dataset/CarCrash/videos/Normal/001355.mp4
1,negative/002192.npz,0,002192,Dataset/CarCrash/videos/Normal/002192.mp4
2,negative/002082.npz,0,002082,Dataset/CarCrash/videos/Normal/002082.mp4
3,negative/000754.npz,0,000754,Dataset/CarCrash/videos/Normal/000754.mp4
4,positive/000713.npz,1,000713,Dataset/CarCrash/videos/Crash-1500/000713.mp4


In [23]:
df.label.value_counts()

label
0    3000
1    1500
Name: count, dtype: int64

In [24]:
df=df.merge(annotation_df_reduced, left_on=["video_file","label"],right_on=["vid","accident"], how="left")

In [25]:
df[df["accident"]==1].accident_frame.describe()


count    1500.000000
mean       37.189333
std         5.787557
min        30.000000
25%        32.000000
50%        36.000000
75%        42.000000
max        49.000000
Name: accident_frame, dtype: float64

In [26]:
df

,npz_file,label_x,video_file,video_path,vid,label_y,accident_frame,ego_involve,accident
0,negative/001355.npz,0,001355,Dataset/CarCrash/videos/Normal/001355.mp4,NaN,NaN,NaN,NaN,NaN
1,negative/002192.npz,0,002192,Dataset/CarCrash/videos/Normal/002192.mp4,NaN,NaN,NaN,NaN,NaN
2,negative/002082.npz,0,002082,Dataset/CarCrash/videos/Normal/002082.mp4,NaN,NaN,NaN,NaN,NaN
3,negative/000754.npz,0,000754,Dataset/CarCrash/videos/Normal/000754.mp4,NaN,NaN,NaN,NaN,NaN
4,positive/000713.npz,1,000713,Dataset/CarCrash/videos/Crash-1500/000713.mp4,000713,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",41.0,No,1.0
...,...,...,...,...,...,...,...,...,...
4495,negative/002476.npz,0,002476,Dataset/CarCrash/videos/Normal/002476.mp4,NaN,NaN,NaN,NaN,NaN
4496,positive/000561.npz,1,000561,Dataset/CarCrash/videos/Crash-1500/000561.mp4,000561,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",39.0,Yes,1.0
4497,negative/001092.npz,0,001092,Dataset/CarCrash/videos/Normal/001092.mp4,NaN,NaN,NaN,NaN,NaN
4498,negative/001327.npz,0,001327,Dataset/CarCrash/videos/Normal/001327.mp4,NaN,NaN,NaN,NaN,NaN


In [27]:
df.drop(["vid","label_x","label_y"],axis=1,inplace=True)
df.accident.fillna(0,inplace=True)
df.ego_involve.fillna("Yes",inplace=True)
df.accident_frame.fillna(-1,inplace=True)
#on ne garde que si ego involve est vrai
df=df[df["ego_involve"]=="Yes"]

df.head()

,npz_file,video_file,video_path,accident_frame,ego_involve,accident
0,negative/001355.npz,001355,Dataset/CarCrash/videos/Normal/001355.mp4,-1.0,Yes,0.0
1,negative/002192.npz,002192,Dataset/CarCrash/videos/Normal/002192.mp4,-1.0,Yes,0.0
2,negative/002082.npz,002082,Dataset/CarCrash/videos/Normal/002082.mp4,-1.0,Yes,0.0
3,negative/000754.npz,000754,Dataset/CarCrash/videos/Normal/000754.mp4,-1.0,Yes,0.0
5,negative/000847.npz,000847,Dataset/CarCrash/videos/Normal/000847.mp4,-1.0,Yes,0.0


In [28]:
df_for_training=df.drop(["accident","ego_involve","video_path","video_file"],axis=1)
df_for_training.head()

,npz_file,accident_frame
0,negative/001355.npz,-1.0
1,negative/002192.npz,-1.0
2,negative/002082.npz,-1.0
3,negative/000754.npz,-1.0
5,negative/000847.npz,-1.0


In [114]:
#save the dataset for training
df_for_training.to_parquet("Dataset/CarCrash/df_for_training.parquet")

In [29]:
#split the dataset into train and test

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_for_training.drop(["accident_frame"],axis=1), df_for_training["accident_frame"], test_size=0.2, random_state=42)

display(X_train.shape)
display(X_test.shape)
display(y_train.shape)
display(y_test.shape)

(3040, 1)

(761, 1)

(3040,)

(761,)

In [ ]:
def read_features_file(file):
    #Code to open and read feature in npz files
    features = np.load(file)  
    #keep only data and det from feature
    features = features["data"]

    return features

In [31]:
X_train

,npz_file
1024,negative/001325.npz
873,negative/000034.npz
601,negative/002041.npz
1947,negative/002490.npz
1475,negative/000685.npz
...,...
1347,negative/001643.npz
1537,positive/000053.npz
1022,negative/000484.npz
4157,negative/002574.npz


In [33]:
test_file="Dataset/CarCrash/vgg16_features/positive/000004.npz"
features = np.load(test_file)


In [36]:
features["data"].shape

(50, 20, 4096)

In [37]:
features["det"].shape

(50, 19, 6)

- data: Extracted 4096-dim features with shape (50, 20, 4096). It contains frame-level feature with shape (50, 1, 4096) and 19 box-level features with shape (50, 19, 4096).
- det: Detected bounding boxes with shape (50, 19, 6), where the last dim denotes (x1, y1, x2, y2, prob, cls).
- labels: One-hot video labels to indicate whether the video contains an accident, i.e., [0, 1] denotes positive (accident) and [1, 0] denotes negative (normal).
- ID: The video name for current feature file.

In [71]:
def prepare_npz_file(file):
    #Code to open and read feature in npz files
    features = np.load(file)  
    #keep only data and det from feature
    features_det = features["det"]  
   
    return features_det

In [104]:
X_train_prep=X_train[:100]
y_train_prep=y_train[:100]
display(X_train_prep)
display(y_train_prep)

,npz_file
1024,negative/001325.npz
873,negative/000034.npz
601,negative/002041.npz
1947,negative/002490.npz
1475,negative/000685.npz
...,...
2833,negative/002540.npz
3166,negative/000796.npz
3246,negative/000868.npz
1908,negative/000765.npz


1024    -1.0
873     -1.0
601     -1.0
1947    -1.0
1475    -1.0
        ... 
2833    -1.0
3166    -1.0
3246    -1.0
1908    -1.0
727     38.0
Name: accident_frame, Length: 100, dtype: float64

In [105]:
#On rrécupère les features
X_train_prep= X_train_prep["npz_file"].apply(lambda  x : prepare_npz_file("Dataset/CarCrash/vgg16_features/"+ x)).to_list()

In [106]:
#On transforme tout en array
X_train_prep=np.array(X_train_prep)

In [107]:
#on prend les 30 première frames
X_train_prep=X_train_prep[:,:30,:]
X_train_prep.shape

(100, 30, 19, 6)

In [108]:
y_train_prep=np.array(y_train_prep.to_list())
y_train_prep

array([-1., -1., -1., -1., -1., 44., -1., 46., -1., -1., 47., 35., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., 48., -1., -1., -1.,
       -1., -1., 45., 36., -1., -1., -1., -1., -1., 32., -1., -1., -1.,
       -1., -1., -1., 35., -1., -1., 46., -1., -1., -1., -1., -1., 48.,
       -1., -1., -1., 30., -1., -1., -1., -1., -1., -1., 40., -1., -1.,
       -1., 35., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., 43., 46., 37., -1., -1., -1., -1., 38.])

In [109]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, SimpleRNN, Dense, TimeDistributed, Flatten


In [110]:
# Créer le modèle
model = Sequential()

# Ajouter des couches Conv2D et MaxPooling2D
# Remarque : Tu dois choisir le nombre et la taille des filtres en fonction de tes données
# Première couche Conv2D et MaxPooling2D
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(30, 19, 6, 1)))
model.add(TimeDistributed(MaxPooling2D(2, 2)))

# Deuxième couche Conv2D avec un filtre plus petit ou padding
model.add(TimeDistributed(Conv2D(64, (2, 2), activation='relu', padding='same'))) # J'ai modifié la taille du filtre et ajouté le padding
model.add(TimeDistributed(MaxPooling2D(2, 2)))

model.add(TimeDistributed(Flatten()))

# Reshape pour le RNN
model.add(Reshape((30, -1)))  # -1 signifie que cette dimension sera calculée automatiquement

# Ajouter des couches RNN
model.add(SimpleRNN(50, activation='relu', return_sequences=True))
model.add(SimpleRNN(50, activation='relu'))

# Couche de sortie
model.add(Dense(2, activation='softmax')) 

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [111]:
y_train_prep=pd.DataFrame(y_train_prep)
y_train_prep.replace(-1,0,inplace=True)
#remplacer toute les valeurs différentes de 0 par 1
y_train_prep[y_train_prep!=0]=1
y_train_prep

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
95,0.0
96,0.0
97,0.0
98,0.0


In [112]:
from tensorflow.keras.utils import to_categorical

# Supposons que y_train soit tes étiquettes originales
y_train_one_hot = to_categorical(y_train_prep, num_classes=2)
y_train_one_hot

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [113]:

# Entraîner le modèle
model.fit(X_train_prep, y_train_one_hot, epochs=10)

Epoch 1/10
4/4 [==============================] - 1s 21ms/step - loss: 98.8753 - accuracy: 0.4500
Epoch 2/10
4/4 [==============================] - 0s 18ms/step - loss: 34.8127 - accuracy: 0.8200
Epoch 3/10
4/4 [==============================] - 0s 16ms/step - loss: 16.6220 - accuracy: 0.8200
Epoch 4/10
4/4 [==============================] - 0s 16ms/step - loss: 20.6435 - accuracy: 0.4200
Epoch 5/10
4/4 [==============================] - 0s 18ms/step - loss: 8.8304 - accuracy: 0.8100
Epoch 6/10
4/4 [==============================] - 0s 16ms/step - loss: 6.2533 - accuracy: 0.8300
Epoch 7/10
4/4 [==============================] - 0s 17ms/step - loss: 4.2432 - accuracy: 0.6900
Epoch 8/10
4/4 [==============================] - 0s 20ms/step - loss: 3.0662 - accuracy: 0.8000
Epoch 9/10
4/4 [==============================] - 0s 20ms/step - loss: 2.8013 - accuracy: 0.8400
Epoch 10/10
4/4 [==============================] - 0s 19ms/step - loss: 3.8433 - accuracy: 0.6200
